# Machine learning homework7

## 0 Preparation

### 0.1 Import librarys

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### 0.2 Read data from files

In [ ]:
mnist_x = np.loadtxt('data/mnist_X.csv', delimiter=',')
mnist_y = np.loadtxt('data/mnist_label.csv', delimiter=',')

## 1 Different dimension reduction methods

In [ ]:
def dim_reduce(x, y, method, need_y=False, title=''):
    """Use specific method to reduce dimension to 2 and plot result."""
    x_reduce = method(x, y, reduce_to=2) if need_y else method(x, reduce_to=2)
    fig, ax = plt.subplots()
    fig.suptitle(title)
    ax.scatter(x_reduce[:, 0], x_reduce[:, 1], c=y)
    plt.show()

### 1.1 PCA (Principal Component Analysis)

In [ ]:
def pca(x, reduce_to=2):
    """Perform PCA on dataset x, reduce to a certain dimension."""
    # Compute convariance matrix
    x_cov = np.cov(x, rowvar=False)
    # Compute eigen value and eigen vector of convariance
    eigen_val, eigen_vec = np.linalg.eig(x_cov)
    # Sort eigen value and vector according to eigen value in descending order
    eigen_vec = eigen_vec[:, np.argsort(eigen_val)[::-1]]
    eigen_val = eigen_val[np.argsort(eigen_val)[::-1]]
    # Project original data to new space
    result = x @ eigen_vec[:, :reduce_to]
    return result.real

In [ ]:
dim_reduce(mnist_x, mnist_y, pca, title='PCA without standardization')

In [ ]:
def standardize(x):
    """Standardize x, resulting in zero mean and unit variance."""
    z = np.zeros_like(x)
    std_x = x.std(axis=0)
    std_x_gt0 = std_x > 0
    z[:, std_x_gt0] = (x - x.mean(axis=0))[:, std_x_gt0] / std_x[std_x_gt0]
    return z

In [ ]:
dim_reduce(standardize(mnist_x), mnist_y, pca, title='PCA with standardization')

### 1.2 LDA (Linear Discriminant Analysis)

Reference:
- https://sebastianraschka.com/Articles/2014_python_lda.html

In [ ]:
def lda(x, y, reduce_to=2):
    """Perform LDA on dataset x, reduce to a certain dimension."""
    labels = np.unique(y)
    n_dim = x.shape[1]
    s_w = np.zeros((n_dim, n_dim))
    s_b = np.zeros_like(s_w)
    total_mean = x.mean(axis=0)
    for label in labels:
        mean = x[y==label].mean(axis=0)
        # Compute within-class scatter matrix
        s_w += np.cov(x[y==label], rowvar=False)
        # Compute between-class scatter matrix
        to_total_mean = mean - total_mean
        s_b += (to_total_mean[None, :] * to_total_mean[:, None]) * (y==label).sum()
    # Get eigen value and eigen vector from (s_w)^-1 * s_b
    eigen_val, eigen_vec = np.linalg.eig(np.linalg.pinv(s_w) @ s_b)
    # Sort eigen value and vector according to eigen value in descending order
    eigen_vec = eigen_vec[:, np.argsort(eigen_val)[::-1]]
    eigen_val = eigen_val[np.argsort(eigen_val)[::-1]]
    # Project original data to new space
    result = x @ eigen_vec[:, :reduce_to]
    return result.real

In [ ]:
dim_reduce(mnist_x, mnist_y, lda, need_y=True, title='LDA without standardization')

In [ ]:
dim_reduce(standardize(mnist_x), mnist_y, lda, need_y=True, title='LDA with standardization')

### 1.3 Symmetric SNE and T-SNE (Stochastic Neighbor Embedding)

## 2 Eigen face